# Hyperparameter Optimization Of TemporalFusionTransformer

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
import numpy as np

In [ ]:
# Load data of closing transaction
df = pd.read_csv('../CSV/df_TS_week.csv')
df.head()

In [ ]:
df["log_target"] = np.log(df.target + 1e-8)
df.head()

In [ ]:
# Drop row where item_id is IT01
df = df[df['item_id'] != 'IT01']

In [ ]:
prediction_length = 12  # Length of the prediction

# Convert the 'timestamp' column to datetime format if it's not already in that format
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Find the maximum date in the dataset
max_date = df['timestamp'].max()

# Calculate the start date of the test set
test_start_date = max_date - pd.DateOffset(weeks=prediction_length+1)

# Split the dataset into training and test sets
train_data = df[df['timestamp'] <= test_start_date]
test_data = df[df['timestamp'] > test_start_date]

# Print the percentage dimensions of the training and test sets
print(train_data.shape[0] / df.shape[0])
print(test_data.shape[0] / df.shape[0])


In [ ]:
# Convert train_data to TimeSeriesDataFrame
train_data = TimeSeriesDataFrame(
    train_data, 
    id_column="item_id",
    timestamp_column="timestamp"
    
)

train_data.head()

In [ ]:
# Convert test_data to TimeSeriesDataFrame
test_data = TimeSeriesDataFrame(
    test_data, 
    id_column="item_id",
    timestamp_column="timestamp"
    
)

test_data.head()

In [ ]:
# Build predictor
predictor = TimeSeriesPredictor(
    prediction_length=12,
    freq='W-MON',
    target='target',
    path='C:\\Develop\\HPO\\ALL_company_HPO',
    eval_metric='MASE'
)

In [ ]:
from autogluon.common import space

predictor.fit(
    train_data,
    hyperparameters = {
        "TemporalFusionTransformer": { 
           "hidden_dim": space.Categorical(32, 64, 128),  
           "variable_dim": space.Categorical(32, 64, 128),
           "num_heads": space.Categorical(1,2,4),
           "dropout_rate": space.Real(0.1, 0.3),
           "max_epochs": space.Categorical(50,100, 150),
           "batch_size": space.Categorical(64, 128, 156),
           "lr": space.Categorical(0.1, 0.01, 0.001),
           "early_stopping_patience" : 10
        },
    },
    hyperparameter_tune_kwargs="auto",
    refit_full=True,
    num_val_windows=3,
    random_seed=42,
    enable_ensemble = False
)

In [ ]:
# Make prediction on training data
predictions = predictor.predict(train_data, random_seed=42)
predictions.head()

In [ ]:
predictor.leaderboard(test_data)

In [ ]:
plt.figure(figsize=(20, 5))
plt.title("Comparison between values predicted by the model and actual values from test data (IT01)", pad=20, fontsize=15)

item_id = "FR13"
y_past = train_data.loc[item_id]["target"]
y_pred = predictions.loc[item_id]
y_test = test_data.loc[item_id]["target"][:12]

# Get the dates for each observation
all_dates = list(y_past.index[-8:]) + list(y_test.index)
all_ticks = range(len(all_dates))

# Set the x-axis range for past and future
past_ticks = range(8)
future_ticks = range(8, len(all_dates))

plt.plot(past_ticks, y_past[-8:], label="Past time series values", marker='o', linestyle='-')
plt.plot(future_ticks, y_pred["mean"], label="Mean forecast", marker='o', linestyle='-')
plt.plot(future_ticks, y_test, label="Future time series values", marker='o', linestyle='-')

# Set the x-axis range for fill_between
fill_ticks = range(8, len(y_pred.index) + 8)

#plt.fill_between(
#    fill_ticks, y_pred["0.1"], y_pred["0.9"], color="red", alpha=0.1, label=f"10%-90% confidence interval"
#)

# Set x-axis labels with dates without time
date_labels = [date.strftime("%Y-%m-%d") for date in all_dates]
plt.xticks(all_ticks, date_labels, rotation=45)

# Add label for x-axis and y-axis
plt.xlabel("Date", fontsize=15, labelpad=15)
plt.ylabel("Number of transactions", fontsize=15, labelpad=15)

plt.legend()
plt.show()

In [ ]:
# Print all row of predictions where item_id is IT01
predictions.loc['IT01']

In [ ]:
# Sum all value of mean forecast of predictions where item_id is IT01
predictions.loc['IT01']['mean'].sum()

In [ ]:
# Sum all value of target of test_data 
test_data.loc['IT01']['target'][:12].sum()

In [ ]:
# Calculate difference betweend each prediction and actual value of test_data
diff = predictions.loc['IT01']['mean'] - test_data.loc['IT01']['target'][:12]
diff

In [ ]:
# Show model names
predictor.model_names()

In [ ]:
# Show parameters of the model
model = predictor._trainer.load_model("TemporalFusionTransformer\\05c5e857")  # provide the model name here
model.get_user_params()

In [ ]:
# Show parameters of the model
model = predictor._trainer.load_model("TemporalFusionTransformer\\69bdc396")  # provide the model name here
model.get_user_params()

In [ ]:
# Show parameters of the model
model = predictor._trainer.load_model("TemporalFusionTransformer\\735a71a3")  # provide the model name here
model.get_user_params()

In [ ]:
# Show parameters of the model
model = predictor._trainer.load_model("TemporalFusionTransformer\\c3fe6316")  # provide the model name here
model.get_user_params()